This notebook is in continuation with **DataEngineeringUsingApacheSparkPart1** Notebook.

# Reading data and importing packages

In [3]:
listing_df = spark.table("listing") 
reviews_df = spark.table("reviews")
neighbourhood_df = spark.table("neighbourhood")

In [4]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from pyspark.sql.functions import when, count, isnan, col

# 10. Customers who travel the most based on reviews data

In [6]:
reviews_df.groupBy("reviewer_id").count().orderBy(col("count").desc()).show(5)

+-----------+-----+
reviewer_id|count|
+-----------+-----+
 86126627| 107|
 111293458| 104|
 71164558| 79|
 563572| 79|
 63643121| 65|
+-----------+-----+
only showing top 5 rows

# 11. first and last time a reviewer gave review

In [8]:
reviews_df.groupBy("reviewer_id").agg(f.min("date").alias("min"), f.max("date").alias("max")).show(5)

+-----------+-------------------+-------------------+
reviewer_id| min| max|
+-----------+-------------------+-------------------+
 150837889|2018-06-12 00:00:00|2018-06-12 00:00:00|
 12661120|2017-06-17 00:00:00|2017-06-17 00:00:00|
 223475|2010-12-24 00:00:00|2010-12-24 00:00:00|
 110055350|2018-05-19 00:00:00|2018-05-19 00:00:00|
 258487|2010-11-11 00:00:00|2010-11-11 00:00:00|
+-----------+-------------------+-------------------+
only showing top 5 rows

In [9]:
reviews_df.groupBy("reviewer_id").agg(f.min("date").alias("min"), f.max("date").alias("max")).orderBy("min", "max", ascending=True).show(5)

+-----------+-------------------+-------------------+
reviewer_id| min| max|
+-----------+-------------------+-------------------+
 2865|2008-09-22 00:00:00|2008-09-22 00:00:00|
 8512|2009-03-12 00:00:00|2009-03-12 00:00:00|
 8096|2009-03-28 00:00:00|2009-03-28 00:00:00|
 9703|2009-04-02 00:00:00|2009-04-02 00:00:00|
 10042|2009-04-07 00:00:00|2009-04-07 00:00:00|
+-----------+-------------------+-------------------+
only showing top 5 rows

# 12. Difference between the maximum and minimum date reviewed. i.e., time difference between latest and oldest review and ordered by number of reviews.

In [11]:
from pyspark.sql.functions import datediff

In [12]:
# Method 1
reviews_df.groupBy("reviewer_id").agg(f.min("date").alias("min"), f.max("date").alias("max"), f.count("reviewer_id").alias("count")).withColumn("datediff", datediff("max", "min")).orderBy("count", ascending=False).show(5)

+-----------+-------------------+-------------------+-----+--------+
reviewer_id| min| max|count|datediff|
+-----------+-------------------+-------------------+-----+--------+
 86126627|2016-09-15 00:00:00|2020-03-09 00:00:00| 107| 1271|
 111293458|2017-04-26 00:00:00|2020-02-26 00:00:00| 104| 1036|
 71164558|2017-09-22 00:00:00|2020-02-28 00:00:00| 79| 889|
 563572|2016-06-15 00:00:00|2019-09-05 00:00:00| 79| 1177|
 63643121|2018-02-01 00:00:00|2020-03-05 00:00:00| 65| 763|
+-----------+-------------------+-------------------+-----+--------+
only showing top 5 rows

In [13]:
# Method2: same as above just with some intermediate stops and then join
df_reviews_cnt = reviews_df.groupBy("reviewer_id").count()
df_reviews_dt = reviews_df.groupBy("reviewer_id").agg(f.min("date").alias("min"), f.max("date").alias("max"))

df_reviews_cnt.join(df_reviews_dt, "reviewer_id").withColumn("datediff", datediff("max", "min")).orderBy("count", ascending=False).show(5)

+-----------+-----+-------------------+-------------------+--------+
reviewer_id|count| min| max|datediff|
+-----------+-----+-------------------+-------------------+--------+
 86126627| 107|2016-09-15 00:00:00|2020-03-09 00:00:00| 1271|
 111293458| 104|2017-04-26 00:00:00|2020-02-26 00:00:00| 1036|
 563572| 79|2016-06-15 00:00:00|2019-09-05 00:00:00| 1177|
 71164558| 79|2017-09-22 00:00:00|2020-02-28 00:00:00| 889|
 63643121| 65|2018-02-01 00:00:00|2020-03-05 00:00:00| 763|
+-----------+-----+-------------------+-------------------+--------+
only showing top 5 rows

# 13. Most frequent travellers based on number of reviews and time taken for those reviews

In [15]:
# Continuing from question 12
# Filter with col("count") > 30 is used because without that there were too many 
# Reviewers with just 1 reviews
df_reviews_cnt.join(df_reviews_dt, "reviewer_id").withColumn("datediff", datediff("max", "min")).withColumn("avg_days", f.round(col("datediff")/col("count"),3)).filter(col("count") > 30).orderBy(col("avg_days").asc()).show(5)

+-----------+-----+-------------------+-------------------+--------+--------+
reviewer_id|count| min| max|datediff|avg_days|
+-----------+-----+-------------------+-------------------+--------+--------+
 89996612| 53|2018-11-16 00:00:00|2020-02-01 00:00:00| 442| 8.34|
 151102716| 35|2017-10-14 00:00:00|2018-08-16 00:00:00| 306| 8.743|
 63296090| 32|2019-05-09 00:00:00|2020-02-13 00:00:00| 280| 8.75|
 132531422| 32|2019-04-03 00:00:00|2020-01-17 00:00:00| 289| 9.031|
 111293458| 104|2017-04-26 00:00:00|2020-02-26 00:00:00| 1036| 9.962|
+-----------+-----+-------------------+-------------------+--------+--------+
only showing top 5 rows